In [1]:

# install the following packages using pip3: 
#
# pip3 install pandas numpy shapely
#
# you will also need pcbflow, which is found in https://github.com/michaelgale/pcbflow

import numpy as np, pandas as pd
from shapely import *
import sys
# change this to the location where pcbflow is located
sys.path.append('/Users/spaul/pcbflow/') 
from pcbflow import *
from pcbflow.route import Route


In [2]:

import os
path = "mypcb"
# Check whether the specified path exists or not
isExist = os.path.exists(path)
if not isExist:
   # Create a new directory because it does not exist
   os.makedirs(path)

In [4]:
def hxrt(A):
    return np.sqrt(A/(3*np.sqrt(3)/2))

In [5]:
from pcbflow import *

def make_board(layer, side):
    df=pd.read_csv(f"insert_layout/PCB_edges_{layer}{side}.csv")
    
    
    
    x=-df.x*10
    y=df.y*10
    #print(x,y)
    height=max(y)-min(y)
    width=max(x)-min(x)
    brd = Board((width, height))
    #standard sized vias
    #brd.drc.via_drill = 0.3
    #brd.drc.via_annular_ring = 0.15
    
    #small vias
    brd.drc.via_drill = 0.1
    brd.drc.via_annular_ring = 0.05
    
    #brd.add_outline()
    offset=[np.min(x), np.min(y)]
    x-=np.min(x)
    y-=np.min(y)
    brd.layers["GML"].lines.append(LinearRing(zip(x,y)))
    brd.add_inner_copper_layer(4 if layer<=16 and side=="R" and layer%4 in (1,2) else 2)
    #help(brd)
    brd.body()
    
    return brd, offset
# fill the top and bottom copper layers and merge nets named "GND"
#brd.fill_layer("GTL", "GND")
#brd.fill_layer("GBL", "GND")

# save the PCB asset files
brd,offset = make_board(1,"L")
brd.save("test_outline")

Rendering Gerber GTD...
Rendering Gerber GTP...
Rendering Gerber GTO...
Rendering Gerber GTS...
Rendering Gerber GTL...
Rendering Gerber GBL...
Rendering Gerber GBS...
Rendering Gerber GBO...
Rendering Gerber GBP...
Rendering Gerber GBD...
Rendering Gerber GML...
Rendering Gerber GP2...
Rendering Gerber GP3...
Rendering excellon drill files...
Rendering preview_top.['pdf']...
Rendering preview_top_docu.['pdf']...
Rendering preview_bot.['pdf']...
Rendering preview_bot_docu.['pdf']...
Rendering preview_all.['pdf']...


In [6]:
def add_documentation(brd, layer, side, layer_info=None, 
                      SiPM="S14160-1315PS", 
                      capacitor="AGC0603X7R101-103KNP (10 nF)",
                      connector=None, version="0.0"):
    if connector==None:
        nconn=2
        if side=="L" and layer<=16:
            connector="HSEC8-130-01-l-S-DV-A"
        elif side=="L" and layer>16:
            connector="HSEC8-120-01-l-S-DV-A"
        elif side=="R" and layer<=16:
            if layer %4 in (0,3):
                connector="HSEC8-170-01-l-S-DV-A"
            else :
                connector="HSEC8-130-01-l-S-DV-A" 
                nconn=4
        elif side=="R" and layer>16:
            connector="HSEC8-130-01-l-S-DV-A"    
    if layer_info is None:
        layer_info=f"{layer}"
    df=pd.read_csv(f"insert_layout/SiPM_positions_{layer}{side}.csv")
    s=\
    f"""CALI PCB board {side}{layer_info}
minimum space between channels={brd.drc.channel():.4f} mm
From top to bottom in each dimple: SiPM, capacitor
SiPM={SiPM} x{len(df)}
capacitor={capacitor} x{len(df)}
connector={connector} x{nconn}
# of cells={len(df)}
cell side length={hxrt(df.area[0]):.2f} cm
cell area={df.area[0]:.1f} cm2"""
    if version is not None:
        s+=f"\nversion {version}"
    #s="hi"
    #print(len(s.split("\n")))
    for i, line in enumerate(s.split("\n")):
        #print(i)
        brd.add_text((brd.size[0]-30,brd.size[1]-5-i*1.5), line,layer='GBO', side='bottom', justify='right')

In [7]:
def add_logos(brd, side):
    if side=="R":
        brd.add_bitmap((90,50),fn="logos/epic.png", scale=2.5, side='bottom', layer='GBO')
        brd.add_bitmap((brd.size[0]-65,70),fn="logos/CALI.png", scale=2.5, side='bottom', layer='GBO')
        brd.add_bitmap((100,brd.size[1]-40),fn="logos/UCR.png", scale=4, side='bottom', layer='GBO')
        brd.add_bitmap((100,brd.size[1]-90),fn="logos/consortium.png", scale=4, side='bottom', layer='GBO')
    else:
        brd.add_bitmap((100,60),fn="logos/epic.png", scale=2.7, side='bottom', layer='GBO')
        brd.add_bitmap((70,brd.size[1]/2-20),fn="logos/CALI.png", scale=3, side='bottom', layer='GBO')
        brd.add_bitmap((90,brd.size[1]*3/4+35),fn="logos/UCR.png", scale=3, side='bottom', layer='GBO')
        brd.add_bitmap((90,brd.size[1]*3/4-15),fn="logos/consortium.png", scale=3, side='bottom', layer='GBO')
brd,offset = make_board(60,"L")
add_logos(brd, "R")
brd.save("test_sipm")
print('saved')

Rendering Gerber GTD...
Rendering Gerber GTP...
Rendering Gerber GTO...
Rendering Gerber GTS...
Rendering Gerber GTL...
Rendering Gerber GBL...
Rendering Gerber GBS...
Rendering Gerber GBO...
Rendering Gerber GBP...
Rendering Gerber GBD...
Rendering Gerber GML...
Rendering Gerber GP2...
Rendering Gerber GP3...
Rendering excellon drill files...
Rendering preview_top.['pdf']...
Rendering preview_top_docu.['pdf']...
Rendering preview_bot.['pdf']...
Rendering preview_bot_docu.['pdf']...
Rendering preview_all.['pdf']...
saved


In [8]:
def outline_polygon(brd, xy):
    dc=brd.DC(xy[0])
    dc.push()
    dc.newpath()
    olddir=90
    for j in range(len(xy)-1):
        #help(dc.goto)    
        t=xy[j+1][1]-xy[j][1],xy[j+1][0]-xy[j][0]
        newdir=180/np.pi*np.arctan2(*t)
        dc.left(newdir-olddir)
        dc.forward(np.hypot(xy[j+1][1]-xy[j][1],xy[j+1][0]-xy[j][0]))
        olddir=newdir
    dc.silk('top')
    dc.pop()

In [9]:
def add_marker(brd, xy):
    outline_polygon(brd, ((xy[0]-.1,xy[1]-.1),(xy[0]+.1,xy[1]+.1)))
    outline_polygon(brd, ((xy[0]+.1,xy[1]-.1),(xy[0]-.1,xy[1]+.1)))

In [10]:
def add_mill_marks(brd, layer, side, additional_layers):
    print("adding mill marks")
    for lay in additional_layers:
        df=pd.read_csv(f"insert_layout/PCB_edges_{lay}{side}.csv")
        x=-df.x*10
        y=df.y*10
        
        x-=np.min(x)
        y-=np.min(y)
        #print("mill mark outline", x,y)
        outline_polygon(brd,list(zip(x,y)))

In [11]:
import glob
def rename_files(folder_name):
    basename=folder_name+"/"+folder_name.split("/")[-1]
    #!diff pcb_output/60L/60L_bot.GBR pcb_output/60L/60L.GTP
    #print(basename)
    os.rename(f"{basename}.GML",f"{basename}-EdgeCuts.gbr")
    i=2
    while True:
        if len(glob.glob(f"{basename}.GP{i}"))==0:
            break
        os.rename(f"{basename}.GP{i}", f"{basename}-In{i-1}_Cu.gbr")
        i+=1

In [12]:
# assigns the layers in the pcb board for the traces going to each SiPM
def choose_trace_layers(df, layer, side):
    if not (layer%4 in (1,2) and layer <=16 and side =="R"):
        df['tracelayer_C']=["GP2"]*len(df)
        df['tracelayer_A']=["GP3"]*len(df)
    else:
        tl=[i%2 for i in range(len(df))]
        df['tracelayer_C']=["GP2" if tl[i]==1 else "GP4" for i in range(len(df))]
        df['tracelayer_A']=["GP3" if tl[i]==1 else "GP5" for i in range(len(df))]

In [13]:
#adds the SiPMs plus associated electronics and traces.  
def add_SiPMs(brd, layer, side, offset):
    df=pd.read_csv(f"insert_layout/SiPM_positions_{layer}{side}.csv")
    choose_trace_layers(df, layer, side)
    x=-df.x*10-offset[0]
    y=df.y*10-offset[1]
    
    df["x_mm"]=x
    df["y_mm"]=y
    
    r=hxrt(df.area[0]*100)
    #create a hexagon
    phi=np.linspace(0, 2*np.pi, 7)
    
    tracelengths={i:0 for i in range(len(df))}
    
    #add these to the dict of layer names
    import pcbflow.kicad as kicad
    kicad.KI_LAYER_DICT["F.CrtYd"]="GTO"
    kicad.KI_LAYER_DICT['"F.SilkS"']=kicad.KI_LAYER_DICT['F.SilkS']
    kicad.KI_LAYER_DICT['"F.Fab"']= kicad.KI_LAYER_DICT['F.Fab']
    kicad.KI_LAYER_DICT['"F.Cu"']= kicad.KI_LAYER_DICT['F.Cu']
    SiPMs=[]
    for i,(xi,yi) in enumerate(zip(x,y)):
        #brd.add_text((xi,yi), "SiPM")
        outline_polygon(brd, list(zip(xi+np.cos(phi)*r,yi+np.sin(phi)*r)))
        #brd.layers["GTS"].lines.append(LinearRing(zip(xi+np.cos(phi)*ri,yi+np.sin(phi)*r)))
        SiPM=KiCadPart(brd.DC((xi, yi)), libraryfile="footprints/SiPM_test_named.kicad_mod")
        SiPMs.append(SiPM)
        brd.add_text((xi,yi-5), f"({xi:.1f}, {yi:.1f}) mm")
        brd.add_text((xi,yi-7), f"({df.row[i]}, {df.col[i]})")
    
    #now make traces towards the inner side for all of the SiPMs
    gap=brd.drc.channel()
    for row in list(set(df.row)):
        for i in range(len(df)):
            if df.row[i]!=row:
                continue
            if side=="L":
                #SiPM cathode
                deltaY=SiPMs[i].pad('"SiPM_C"').xy[1]-SiPMs[i].pad('"GND"').xy[1]+2*gap
                deltaX=max(df.query(f"row=={row}").x_mm)-SiPMs[i].pad('"SiPM_C"').xy[0]+2*gap+3.0
                c=df.col[i]//2
                SiPMs[i].pad('"SiPM_C"').set_layer("GP2").left(90).forward(2*gap).left(90)\
                            .forward(deltaY+(c)*gap).left(90).forward(deltaX).wire()
                
                #SiPM anode
                deltaY=SiPMs[i].pad('"SiPM_A"').xy[1]-SiPMs[i].pad('"GND"').xy[1]+2*gap
                deltaX=max(df.query(f"row=={row}").x_mm)-SiPMs[i].pad('"SiPM_A"').xy[0]+2*gap+3.0
                SiPMs[i].pad('"SiPM_A"').set_layer("GP3").left(90).forward(2*gap).left(90)\
                            .forward(deltaY+(c)*gap).left(90).forward(deltaX).wire()
                tracelengths[i]+=abs(2*gap)+abs(deltaY+(c)*gap)+abs(deltaX)
            else:
                cathode_trace_layer=df.tracelayer_C[i]
                anode_trace_layer=df.tracelayer_A[i]
                #SiPM cathode
                deltaY=SiPMs[i].pad('"SiPM_C"').xy[1]-SiPMs[i].pad('"GND"').xy[1]+2*gap
                deltaX=SiPMs[i].pad('"SiPM_C"').xy[0]-min(df.query(f"row=={row}").x_mm)-1.0+6.0
                #c=(max(df.query(f"row=={row}").col)-df.col[i])//2
                #need to calculate this differently
                c=len(df.query(f"row=={row} and tracelayer_C=='{df.tracelayer_C[i]}' and col>{df.col[i]}"))
                
                SiPMs[i].pad('"SiPM_C"').set_layer(cathode_trace_layer).left(90).forward(1.0).left(90)\
                            .forward(deltaY+(c)*gap).right(90).forward(deltaX).wire()

                #SiPM anode
                deltaY=SiPMs[i].pad('"SiPM_A"').xy[1]-SiPMs[i].pad('"GND"').xy[1]+2*gap
                deltaX=SiPMs[i].pad('"SiPM_A"').xy[0]-min(df.query(f"row=={row}").x_mm)-1.0+6.0
                SiPMs[i].pad('"SiPM_A"').set_layer(anode_trace_layer).left(90).forward(1.0).left(90)\
                            .forward(deltaY+(c)*gap).right(90).forward(deltaX).wire()
                tracelengths[i]+=1.0+abs(deltaY+(c)*gap)+abs(deltaX)
            # have the traces connecting the capacitor to the GND all connect to one another
            # as one another.  Must be higher than the SiPMs' vias
            deltaX=df.x_mm[i]-SiPMs[i].pad('"GND"').xy[0]
            deltaY=5.0#SiPMs[i].xy[1]-SiPMs[i].pad('"GND"').xy[1]+5.0
            SiPMs[i].pad('"GND"').through().left(90).forward(deltaX).right(90)\
                        .forward(deltaY).wire()
            
            if side=="L" and df.col[i]==min(df.query(f"row=={row}").col):
                xGND=10
                a=SiPMs[i].pad('"GND"').copy().right(90).forward(SiPMs[i].pad('"GND"').xy[0]-xGND)
                a.wire()
                if row==min(df.row):
                    a.left(90).forward(max(df.y_mm)-min(df.y_mm)).wire()
                if df.col[i]==min(df.query(f"row=={max(df.row)}").col) and df.row[i]==max(df.row):
                    GND=SiPMs[i].pad('"GND"')
            if side=="R" and df.col[i]==max(df.query(f"row=={row}").col):
                xGND=brd.size[0]-10
                a=SiPMs[i].pad('"GND"').copy().left(90).forward(xGND-SiPMs[i].pad('"GND"').xy[0]).wire()
                if row==min(df.row):
                    a.right(90).forward(max(df.y_mm)-min(df.y_mm)).wire()
            if side=="R" and df.col[i]==min(df.query(f"row=={max(df.row)}").col) and df.row[i]==max(df.row):
                GND=SiPMs[i].pad('"GND"')
    
    return SiPMs, GND, tracelengths


In [14]:
def add_connectors(brd, layer, side, offset, SiPMs, GND, connector_positions, tracelengths):
    df=pd.read_csv(f"insert_layout/SiPM_positions_{layer}{side}.csv")
    choose_trace_layers(df, layer, side)
    x=-df.x*10-offset[0]
    y=df.y*10-offset[1]
    
    df["x_mm"]=x
    df["y_mm"]=y
    sidelength=10*hxrt(df.area[0])
    
    
    
    gap=brd.drc.channel()
    
    nrows=len(set(df.row))
    # repeat this process with only the top group of rows 
    # and then with the bottom ones
    for UD in 1, -1:
        
        
        median_row=np.median(list(set(df.row)))
        selection=(df.row>median_row) if UD==1 else (df.row<=median_row)
        rows=list(sorted(list(set(df.row[selection]))))
        #print(rows)
        if UD==-1: #lower side
            rows=list(reversed(rows))
        #print(rows)
        
        if side=="L":
            
            xconn=brd.size[0]-9
            if layer<=16:
                yconn=brd.size[1]/2+UD*252.5
                connector_name="footprints/Samtec_HSEC8-130-01-X-DV-A_2x30_P0.8mm_Socket_AlignmentPins.kicad_mod"
            else:
                yconn=brd.size[1]/2+UD*270.5
                connector_name="footprints/Samtec_HSEC8-120-01-X-DV-A_2x20_P0.8mm_Socket_AlignmentPins.kicad_mod"
            connector=KiCadPart(brd.DC((xconn, yconn),90), libraryfile=connector_name)
            connector_pads=connector.pads
            if UD==1:
                connector_pads[0].left(180).forward(4).right(90).forward(6).through().forward(GND.xy[1]-connector_pads[0].xy[1]).meet(GND)
                
                
        if side=="R":
            xconn=9
            if layer<=16:
                if layer % 4 in (0,3):
                    yconn=[brd.size[1]/2+UD*222.5]
                    connector_name="footprints/Samtec_HSEC8-170-01-X-DV_2x70_P0.8mm_Pol32_Socket.kicad_mod"
                else:
                    if (UD==1) == ((layer %4)==1):
                        yconn=[brd.size[1]/2+UD*252.5,brd.size[1]/2+UD*215.1]
                    else:
                        yconn=[brd.size[1]/2+UD*233.9,brd.size[1]/2+UD*196.5]
                    connector_name="footprints/Samtec_HSEC8-130-01-X-DV-A_2x30_P0.8mm_Socket_AlignmentPins.kicad_mod"
            else:
                yconn=[brd.size[1]/2+UD*242.0]
                connector_name="footprints/Samtec_HSEC8-130-01-X-DV-A_2x30_P0.8mm_Socket_AlignmentPins.kicad_mod"
            connector_pads=[]
            for y in yconn:
                connector=KiCadPart(brd.DC((xconn, y),90), libraryfile=connector_name)
                if UD==1:
                    connector_pads=connector_pads + list(connector.pads[:-2])
                else: connector_pads=list(connector.pads[:-2])+connector_pads
            if UD==1:
                connector_pads[0].left(180).forward(1).right(90).forward(9.6).right(90).forward(2.6).through().right(90).forward(GND.xy[1]-connector_pads[0].xy[1]).meet(GND)
                        
        #now pair every SiPM trace with a connector trace
        SiPMs_to_connectors={}
        connectors_to_SiPMs={}
        SiPM_to_index={SiPMs[i].pad('"SiPM_A"'):i for i in range(len(SiPMs))}
        for i, pad in enumerate(connector_pads[2 if UD==1 else 0:]):
            if i %2==1:
                #cathode
                #find the SiPM with the uppermost cathode trace that isn't already connected
                ymax_not_connected=-10
                j_at_ymax=-1
                for j,SiPM in enumerate(SiPMs):
                    if selection[j]==False:
                        continue
                    if SiPM.pad('"SiPM_C"') not in SiPMs_to_connectors and SiPM.pad('"SiPM_C"').xy[1]>ymax_not_connected:
                        ymax_not_connected=SiPM.pad('"SiPM_C"').xy[1]
                        j_at_ymax=j
                if j_at_ymax !=-1:
                    SiPMs_to_connectors[SiPMs[j_at_ymax].pad('"SiPM_C"')]=pad
                    connectors_to_SiPMs[pad]=SiPMs[j_at_ymax].pad('"SiPM_C"')
            else:
                #anode
                #find the SiPM with the uppermost anode trace that isn't already connected
                ymax_not_connected=-10
                j_at_ymax=-1
                for j,SiPM in enumerate(SiPMs):
                    if selection[j]==False:
                        continue
                    if SiPM.pad('"SiPM_A"') not in SiPMs_to_connectors and SiPM.pad('"SiPM_A"').xy[1]>ymax_not_connected:
                        ymax_not_connected=SiPM.pad('"SiPM_A"').xy[1]
                        j_at_ymax=j
                if j_at_ymax !=-1:
                    SiPMs_to_connectors[SiPMs[j_at_ymax].pad('"SiPM_A"')]=pad
                    connectors_to_SiPMs[pad]=SiPMs[j_at_ymax].pad('"SiPM_A"')
        
        #first connect the row closest to the top or bottom to their respective connectors
        for j in range(len(SiPMs)):
            if df.row[j]!=rows[-1]:
                continue
            if side=="L":
                for AC in "AC":
                    dy=SiPMs_to_connectors[SiPMs[j].pad(f'"SiPM_{AC}"')].xy[1]-SiPMs[j].pad(f'"SiPM_{AC}"').xy[1]
                    if dy>0:
                        aa=(df.col[j]-min(df.col[df.row==rows[-1]]))//2   
                    else:
                        aa=(max(df.col[df.row==rows[-1]])-df.col[j])//2
                    SiPMs[j].pad(f'"SiPM_{AC}"').forward(gap*aa).left(90*np.sign(dy))\
                        .forward(abs(dy)).wire()
                    if AC=="A":
                        tracelengths[j]+=abs(gap*aa)+abs(dy)
            if side=="R":
                for AC in "AC":
                    if UD==-1 and max(df[df.row==min(df.row)].col) != max(df.col):
                        extension=sidelength*1.5+gap*7 #make sure this doesn't interfere with the row above it
                    elif UD==1 and max(df[df.row==max(df.row)].col) != max(df.col):
                        extension=sidelength*1.5+gap*7 #make sure this doesn't interfere with the row above it
                    else:
                        extension=0
                    dy=SiPMs_to_connectors[SiPMs[j].pad(f'"SiPM_{AC}"')].xy[1]-SiPMs[j].pad(f'"SiPM_{AC}"').xy[1]
#                     if dy<0:
#                         aa=(df.col[j]-min(df.col[df.row==rows[-1]]))//2   
#                     else:
#                         aa=(max(df.col[df.row==rows[-1]])-df.col[j])//2
                    if dy<0:
                        aa=len(df.query(f'col<{df.col[j]} and row=={rows[-1]} and tracelayer_C=="{df.tracelayer_C[j]}"'))
                    else:
                        aa=len(df.query(f'col>{df.col[j]} and row=={rows[-1]} and tracelayer_C=="{df.tracelayer_C[j]}"'))
                    SiPMs[j].pad(f'"SiPM_{AC}"').forward(extension+gap*aa).left(-90*np.sign(dy))\
                        .forward(abs(dy)).wire()
                    if AC=="A":
                        tracelengths[j]+=abs(extension+gap*aa)+abs(dy)
        
        
        # now bunch together the traces from different rows:
        for m in range(1, len(rows)):
            
                            
            for i,row in enumerate(rows[:-m]):
                indices_in_bunch=[]
                for j in range(len(SiPMs)):
                    if df.row[j]==row:
                        indices_in_bunch.append(j)
                #find the y of the uppermost trace in this row
                y_this_row=(min if UD==-1 else max)([SiPMs[l].pad('"SiPM_C"').xy[1] for l in indices_in_bunch])
                for j in indices_in_bunch:
                    if side=="R":
                        dy_to_connector=abs(SiPMs[j].pad('"SiPM_C"').xy[1]-SiPMs_to_connectors[SiPMs[j].pad('"SiPM_C"')].xy[1])
                        dir_to_connector=np.sign(-SiPMs[j].pad('"SiPM_C"').xy[1]+SiPMs_to_connectors[SiPMs[j].pad('"SiPM_C"')].xy[1])
                        if dy_to_connector<0.01:
                            continue
                        deltaX=0
                        deltaY=None
                        y_next_row=None
                        for k in range(len(SiPMs)):
                            if df.tracelayer_C[k]!=df.tracelayer_C[j] and UD==-1:
                                continue
                            dx=SiPMs[j].pad('"SiPM_C"').xy[0]-SiPMs[k].pad('"SiPM_C"').xy[0]
                            #if dx<=0:
                            #    continue
                            if df.row[k] not in rows[i+1:]:
                                continue
                            if (df.row[k]-df.row[j])*UD<=0:
                                continue
                            if df.row[k]!= rows[i+1]:
                                continue

                            if UD>0:
                                y=SiPMs[k].pad('"SiPM_C"').xy[1]
                                if y_next_row is None or y<y_next_row:
                                    y_next_row=y
                                    deltaY=abs(y_this_row-y_next_row)-gap
                                    deltaX=dx
                            else:
                                y=SiPMs[k].pad('"SiPM_C"').xy[1]
                                if y_next_row is None or y>y_next_row:
                                    y_next_row=y
                                    deltaY=abs(y_this_row-y_next_row)-gap
                                    deltaX=dx
                        
                        #gotta be very careful with this patch
#                         if row == min(rows)+1 and UD==-1:
#                             deltaX-=gap*(len(df[df.row==min(rows)])+1)
#                         if row == max(rows)-1 and UD==1:
#                             deltaX-=gap*(len(df[df.row==max(rows)])+1)
                        if row == min(rows)+1 and UD==-1:
                            deltaX-=gap*(len(df.query(f"row=={min(rows)} and tracelayer_C == '{df.tracelayer_C[j]}'"))+1)
                        if row == max(rows)-1 and UD==1:
                            deltaX-=gap*(len(df.query(f"row=={max(rows)} and tracelayer_C == '{df.tracelayer_C[j]}'"))+1)
                        
                        
                        if deltaX<0:
                            deltaX=0
                        if deltaY is None:
                            deltaY=dy_to_connector
                        #don't overshoot
                        if deltaY<0:
                            deltaY=0
                        if deltaY>dy_to_connector:
                            deltaY=dy_to_connector
                            
                        if dir_to_connector*UD<0:
                            deltaY=-dy_to_connector
                        #print("deltaY",deltaY)
                        #print("y",y)
                        
#                         if dir_to_connector==-1:
#                             extension=gap*((df.col[j]-min(df.col[df.row==row]))//2)
#                         elif dir_to_connector==1:
#                             extension=gap*((max(df.col[df.row==row])-df.col[j])//2)
                        if dir_to_connector==-1:
                            extension=gap*len(df.query(f"row=={row} and col<{df.col[j]} and tracelayer_C == '{df.tracelayer_C[j]}'"))
                        elif dir_to_connector==1:
                            extension=gap*len(df.query(f"row=={row} and col>{df.col[j]} and tracelayer_C == '{df.tracelayer_C[j]}'"))
                        
                        #determine how far to extend further
                        aa=len(df.query(f"row=={row} and tracelayer_C == '{df.tracelayer_C[j]}'"))
                        #if 
                        #    aa+=1
                        if deltaX!=0:
                            for AC in "AC":
                                if not(deltaX>1.2*sidelength) or deltaY<.7*sidelength:
                                    SiPMs[j].pad(f'"SiPM_{AC}"').forward(deltaX+extension).right(UD*90).forward(deltaY).left(UD*90).wire().forward(aa*gap-extension).wire()
                                    if AC=="A":
                                        tracelengths[j]+=abs(deltaX+extension)+abs(deltaY)+abs(aa*gap-extension)
                                else:
                                    if deltaX>6*sidelength:
                                        diagx=sidelength*5.7
                                        diagy=sidelength/2 if UD==-1 else sidelength*.75
                                    elif deltaX>4.3*sidelength:
                                        #include a diagonal segment
                                        diagx=sidelength*4.2
                                        diagy=sidelength/2 if UD==-1 else sidelength*.75
                                    else:
                                        diagx=sidelength*1.1
                                        diagy=sidelength*.5 if UD==-1 else sidelength*.75
                                    theta=UD*180/np.pi*np.arctan(diagy/diagx)
                                    diag=np.hypot(diagx,diagy)
                                    
                                    #print("theta",theta, "diag", diag)
                                    SiPMs[j].pad(f'"SiPM_{AC}"').forward(deltaX+extension-diagx).right(theta).forward(diag).right(UD*90-theta).forward(deltaY-diagy).left(UD*90).wire().forward(aa*gap-extension).wire()
                                    if AC=="A":
                                        tracelengths[j]+=abs(deltaX+extension-diagx)+abs(diag)+abs(deltaY-diagy)+abs(aa*gap-extension)
                        else:
                            for AC in "AC":
                                extend_to_connector=True
                                #now check if it's ok to extend the rest of the way to the connector height
                                for k in range(len(SiPMs)):
                                    if k == j: 
                                        continue
                                    if not selection[k]:
                                        continue
                                    if UD*(df.row[k]-df.row[j])<=0:
                                        continue
                                    if df.tracelayer_C[k] != df.tracelayer_C[j]:
                                        continue
                                    if SiPMs[k].pad(f'"SiPM_{AC}"').xy[0]>SiPMs[j].pad(f'"SiPM_{AC}"').xy[0]+gap:
                                        continue
                                    extend_to_connector=False
                                
                                if extend_to_connector:
                                    deltaY=dy_to_connector
                                SiPMs[j].pad(f'"SiPM_{AC}"').forward(deltaX+extension).right(UD*90).forward(deltaY).left(UD*90).wire().forward(aa*gap-extension).wire()
                                if AC=="A":
                                    tracelengths[j]+=abs(deltaX+extension)+abs(deltaY)+abs(aa*gap-extension)
                        #debug put marker on the silkscreen
                        #add_marker(brd, SiPMs[j].pad('"SiPM_C"').xy)
                        
                    if side=="L":
                        dy_to_connector=abs(SiPMs[j].pad('"SiPM_C"').xy[1]-SiPMs_to_connectors[SiPMs[j].pad('"SiPM_C"')].xy[1])
                        dir_to_connector=np.sign(SiPMs[j].pad('"SiPM_C"').xy[1]-SiPMs_to_connectors[SiPMs[j].pad('"SiPM_C"')].xy[1])
                        if dy_to_connector<0.01:
                            continue
                        deltaX=0
                        deltaY=None
                        y_next_row=None
                        for k in range(len(SiPMs)):
                            dx=SiPMs[k].pad('"SiPM_C"').xy[0]-SiPMs[j].pad('"SiPM_C"').xy[0]
                            #if dx<=0:
                            #    continue
                            if df.row[k] not in rows[i+1:]:
                                continue
                            if (df.row[k]-df.row[j])*UD<=0:
                                continue
                            if df.row[k]!= rows[i+1]:
                                continue

                            if UD>0:
                                y=SiPMs[k].pad('"SiPM_C"').xy[1]
                                if y_next_row is None or y<y_next_row:
                                    y_next_row=y
                                    deltaY=abs(y_this_row-y_next_row)-gap
                                    deltaX=dx
                            else:
                                y=SiPMs[k].pad('"SiPM_C"').xy[1]
                                if y_next_row is None or y>y_next_row:
                                    y_next_row=y
                                    deltaY=abs(y_this_row-y_next_row)-gap
                                    deltaX=dx
                        if(df.row[j]==1 and df.col[j]==0):
                            print(deltaX, deltaY, dy_to_connector)    
                            
                        if deltaX<0:
                            deltaX=0
                        if deltaY is None:
                            deltaY=dy_to_connector
                        #don't overshoot
                        if deltaY<0:
                            deltaY=0
                        if deltaY>dy_to_connector:
                            deltaY=dy_to_connector
                        #print("deltaY",deltaY)
                        #print("y",y)
                        if(df.row[j]==1 and df.col[j]==0):
                            print(deltaX, deltaY, dy_to_connector)
                        
                        if UD==1:
                            extension=gap*((df.col[j]-min(df.col[df.row==row]))//2)
                        elif UD==-1:
                            extension=gap*((max(df.col[df.row==row])-df.col[j])//2)
                        #determine how far to extend further
                        aa=len(df.query(f"row=={rows[i]}"))
                        if deltaX!=0:
                            for AC in "AC":
                                SiPMs[j].pad(f'"SiPM_{AC}"').forward(deltaX+extension).left(UD*90).forward(deltaY).right(UD*90).wire().forward(aa*gap-extension).wire()
                                if AC=="A":
                                    tracelengths[j]+=abs(deltaX+extension)+abs(deltaY)+abs(aa*gap-extension)
                        else:
                            for AC in "AC":
                                extend_to_connector=True
                                #now check if it's ok to extend the rest of the way to the connector height
                                for k in range(len(SiPMs)):
                                    if k == j: 
                                        continue
                                    if not selection[k]:
                                        continue
                                    if UD*(df.row[k]-df.row[j])<=0:
                                        continue
                                    if SiPMs[k].pad(f'"SiPM_{AC}"').xy[0]<SiPMs[j].pad(f'"SiPM_{AC}"').xy[0]-gap:
                                        continue
                                    extend_to_connector=False
                                
                                if extend_to_connector:
                                    deltaY=dy_to_connector
                                SiPMs[j].pad(f'"SiPM_{AC}"').forward(deltaX+extension).left(UD*90).forward(deltaY).right(UD*90).wire().forward(aa*gap-extension).wire()
                                if AC=="A":
                                    tracelengths[j]+=abs(deltaX+extension)+abs(deltaY)+abs(aa*gap-extension)
        
        #next connect every pad from the connector to its anode or cathode trace
        for i, pad in enumerate(connector_pads[2 if UD==1 else 0:]):
            
            if i %2==1:
                #cathode
                if pad in connectors_to_SiPMs:
                    SiPM_C=connectors_to_SiPMs[pad]
                    if side == "L":
                        pad.forward(1.6).through().set_layer(SiPM_C.layer).left(90).forward(0.4).left(90).forward(7).left(90).forward(0.4).wire()
                    elif side=="R":
                        pad.forward(1.6).through().set_layer(SiPM_C.layer).wire()
                    SiPM_C.meet(pad)
                    
            else:
                #anode
                if pad in connectors_to_SiPMs:
                    SiPM_A=connectors_to_SiPMs[pad]
                    if side == "L":
                        pad.right(180).forward(1.6).through().set_layer(SiPM_A.layer).wire()
                        
                    elif side == "R":
                        pad.right(180).forward(1.6).through().set_layer(SiPM_A.layer).right(90).forward(0.4).right(90).forward(7).right(90).forward(0.4).wire()
                    dist=SiPM_A.distance(pad)
                    SiPM_A.meet(pad)
                    tracelengths[SiPM_to_index[SiPM_A]]+=dist
    return tracelengths
                
                
            
#test this method
# layer=1
# brdid=(layer,"R")
# brd,offset = make_board(*brdid)
# #add_mill_marks(brd,*brdid, [layer+4, layer+8, layer+12, layer+16])
# #add_mill_marks(brd,*brdid, [60])
# SiPMs,GND, LED_A=add_SiPMs(brd, *brdid, offset)
# add_documentation(brd, *brdid)
# connector_positions=[]
# add_connectors(brd, *brdid, offset, SiPMs, GND, LED_A, connector_positions)
# brd.save("test_sipm")
# print('saved')

In [15]:
def add_tracelengths(brd, layer, side, offset, tracelengths):
    df=pd.read_csv(f"insert_layout/SiPM_positions_{layer}{side}.csv")
    choose_trace_layers(df, layer, side)
    x=-df.x*10-offset[0]
    y=df.y*10-offset[1]
    
    df["x_mm"]=x
    df["y_mm"]=y
    
    r=hxrt(df.area[0]*100)
    #create a hexagon
    phi=np.linspace(0, 2*np.pi, 7)
    
    #add these to the dict of layer names
    import pcbflow.kicad as kicad
    kicad.KI_LAYER_DICT["F.CrtYd"]="GTO"
    kicad.KI_LAYER_DICT['"F.SilkS"']=kicad.KI_LAYER_DICT['F.SilkS']
    kicad.KI_LAYER_DICT['"F.Fab"']= kicad.KI_LAYER_DICT['F.Fab']
    kicad.KI_LAYER_DICT['"F.Cu"']= kicad.KI_LAYER_DICT['F.Cu']
    for i,(xi,yi) in enumerate(zip(x,y)):
        
        brd.add_text((xi,yi-10), f"tracelength={tracelengths[i]:.0f} mm")

In [16]:
layout_list=pd.read_csv("insert_layout/layouts.csv")

In [ ]:
# now create the boards, one group at a time.  

In [17]:
# first create the left boards:
for i in range(len(layout_list)):
    side=layout_list.side[i]
    if side!="L":
        continue
    #find the highest layer number in the list
    layer=int(layout_list.layers[i].split(";")[-1].split("-")[-1].replace('\'',''))
    brdid=(layer,side)
    brd,offset = make_board(*brdid)
    
    SiPMs,GND,tracelengths=add_SiPMs(brd, *brdid, offset)
    layer_info=layout_list.layers[i].replace('\'','')
    add_documentation(brd, layer,side, layer_info=layer_info)
    connector_positions=[]
    tracelengths=add_connectors(brd, *brdid, offset, SiPMs, GND, connector_positions,tracelengths)
    add_tracelengths(brd, *brdid, offset, tracelengths)
    add_logos(brd, side)
    
    path = f"pcb_output/{layer}{side}"
    # Check whether the specified path exists or not
    isExist = os.path.exists(path)
    if not isExist:
       # Create a new directory because it does not exist
       os.makedirs(path)
    brd.save(path.split('/')[-1], subdir=path.split('/')[0])
    #doesn't save in subfolders.  So put these in the subfolders
    import glob
    for a in list(glob.glob(path+".*"))+list(glob.glob(path+"-*"))+list(glob.glob(path+"_*")):
        os.rename(a, path+"/"+a.split("/")[-1])
    rename_files(f"pcb_output/{layer}{side}")
    print("done with layer ", layer, "side", side)

Rendering Gerber GTD...
Rendering Gerber GTP...
Rendering Gerber GTO...
Rendering Gerber GTS...
Rendering Gerber GTL...
Rendering Gerber GBL...
Rendering Gerber GBS...
Rendering Gerber GBO...
Rendering Gerber GBP...
Rendering Gerber GBD...
Rendering Gerber GML...
Rendering Gerber GP2...
Rendering Gerber GP3...
Rendering excellon drill files...
Rendering preview_top.['pdf']...
Rendering preview_top_docu.['pdf']...
Rendering preview_bot.['pdf']...
Rendering preview_bot_docu.['pdf']...
Rendering preview_all.['pdf']...
done with layer  5 side L
Rendering Gerber GTD...
Rendering Gerber GTP...
Rendering Gerber GTO...
Rendering Gerber GTS...
Rendering Gerber GTL...
Rendering Gerber GBL...
Rendering Gerber GBS...
Rendering Gerber GBO...
Rendering Gerber GBP...
Rendering Gerber GBD...
Rendering Gerber GML...
Rendering Gerber GP2...
Rendering Gerber GP3...
Rendering excellon drill files...
Rendering preview_top.['pdf']...
Rendering preview_top_docu.['pdf']...
Rendering preview_bot.['pdf']...
Ren

In [18]:
#do the same for boards R17-60:
for i in range(len(layout_list)):
    side=layout_list.side[i]
    if side!="R":
        continue
    #find the highest layer number in the list
    layer=int(layout_list.layers[i].split(";")[-1].split("-")[-1].replace('\'',''))
    if layer < 17:
        continue
    brdid=(layer,side)
    brd,offset = make_board(*brdid)
    
    SiPMs,GND,tracelengths=add_SiPMs(brd, *brdid, offset)
    layer_info=layout_list.layers[i].replace('\'','')
    add_documentation(brd, layer,side, layer_info=layer_info)
    connector_positions=[]
    tracelengths=add_connectors(brd, *brdid, offset, SiPMs, GND, connector_positions,tracelengths)
    add_tracelengths(brd, *brdid, offset, tracelengths)
    add_logos(brd, side)
    
    path = f"pcb_output/{layer}{side}"
    # Check whether the specified path exists or not
    isExist = os.path.exists(path)
    if not isExist:
       # Create a new directory because it does not exist
       os.makedirs(path)
    brd.save(path.split('/')[-1], subdir=path.split('/')[0])
    #doesn't save in subfolders.  So put these in the subfolders
    import glob
    for a in list(glob.glob(path+".*"))+list(glob.glob(path+"-*"))+list(glob.glob(path+"_*")):
        os.rename(a, path+"/"+a.split("/")[-1])
    rename_files(f"pcb_output/{layer}{side}")
    print("done with layer ", layer, "side", side)

Rendering Gerber GTD...
Rendering Gerber GTP...
Rendering Gerber GTO...
Rendering Gerber GTS...
Rendering Gerber GTL...
Rendering Gerber GBL...
Rendering Gerber GBS...
Rendering Gerber GBO...
Rendering Gerber GBP...
Rendering Gerber GBD...
Rendering Gerber GML...
Rendering Gerber GP2...
Rendering Gerber GP3...
Rendering excellon drill files...
Rendering preview_top.['pdf']...
Rendering preview_top_docu.['pdf']...
Rendering preview_bot.['pdf']...
Rendering preview_bot_docu.['pdf']...
Rendering preview_all.['pdf']...
done with layer  18 side R
Rendering Gerber GTD...
Rendering Gerber GTP...
Rendering Gerber GTO...
Rendering Gerber GTS...
Rendering Gerber GTL...
Rendering Gerber GBL...
Rendering Gerber GBS...
Rendering Gerber GBO...
Rendering Gerber GBP...
Rendering Gerber GBD...
Rendering Gerber GML...
Rendering Gerber GP2...
Rendering Gerber GP3...
Rendering excellon drill files...
Rendering preview_top.['pdf']...
Rendering preview_top_docu.['pdf']...
Rendering preview_bot.['pdf']...
Re

In [19]:
#do the same for boards R1-16 (3mod4 and 0mod4)
for i in range(len(layout_list)):
    side=layout_list.side[i]
    if side!="R":
        continue
    #find the highest layer number in the list
    layer=int(layout_list.layers[i].split(";")[-1].split("-")[-1].replace('\'',''))
    if layer > 16 or layer %4 not in (0,3):
        continue
    brdid=(layer,side)
    brd,offset = make_board(*brdid)
    
    SiPMs,GND,tracelengths=add_SiPMs(brd, *brdid, offset)
    layer_info=layout_list.layers[i].replace('\'','')
    add_documentation(brd, layer,side, layer_info=layer_info)
    connector_positions=[]
    tracelengths=add_connectors(brd, *brdid, offset, SiPMs, GND, connector_positions,tracelengths)
    add_tracelengths(brd, *brdid, offset, tracelengths)
    add_logos(brd, side)
    
    path = f"pcb_output/{layer}{side}"
    # Check whether the specified path exists or not
    isExist = os.path.exists(path)
    if not isExist:
       # Create a new directory because it does not exist
       os.makedirs(path)
    brd.save(path.split('/')[-1], subdir=path.split('/')[0])
    #doesn't save in subfolders.  So put these in the subfolders
    import glob
    for a in list(glob.glob(path+".*"))+list(glob.glob(path+"-*"))+list(glob.glob(path+"_*")):
        os.rename(a, path+"/"+a.split("/")[-1])
    rename_files(f"pcb_output/{layer}{side}")
    print("done with layer ", layer, "side", side)

Rendering Gerber GTD...
Rendering Gerber GTP...
Rendering Gerber GTO...
Rendering Gerber GTS...
Rendering Gerber GTL...
Rendering Gerber GBL...
Rendering Gerber GBS...
Rendering Gerber GBO...
Rendering Gerber GBP...
Rendering Gerber GBD...
Rendering Gerber GML...
Rendering Gerber GP2...
Rendering Gerber GP3...
Rendering excellon drill files...
Rendering preview_top.['pdf']...
Rendering preview_top_docu.['pdf']...
Rendering preview_bot.['pdf']...
Rendering preview_bot_docu.['pdf']...
Rendering preview_all.['pdf']...
done with layer  3 side R
Rendering Gerber GTD...
Rendering Gerber GTP...
Rendering Gerber GTO...
Rendering Gerber GTS...
Rendering Gerber GTL...
Rendering Gerber GBL...
Rendering Gerber GBS...
Rendering Gerber GBO...
Rendering Gerber GBP...
Rendering Gerber GBD...
Rendering Gerber GML...
Rendering Gerber GP2...
Rendering Gerber GP3...
Rendering excellon drill files...
Rendering preview_top.['pdf']...
Rendering preview_top_docu.['pdf']...
Rendering preview_bot.['pdf']...
Ren

In [20]:
#do the same for boards R1-16 (1mod4 and 2mod4)
for i in range(len(layout_list)):
    side=layout_list.side[i]
    if side!="R":
        continue
    #find the highest layer number in the list
    layer=int(layout_list.layers[i].split(";")[-1].split("-")[-1].replace('\'',''))
    if layer > 16 or layer %4 not in (1,2):
        continue
    brdid=(layer,side)
    brd,offset = make_board(*brdid)
    
    SiPMs,GND,tracelengths=add_SiPMs(brd, *brdid, offset)
    layer_info=layout_list.layers[i].replace('\'','')
    add_documentation(brd, layer,side, layer_info=layer_info)
    connector_positions=[]
    tracelengths=add_connectors(brd, *brdid, offset, SiPMs, GND, connector_positions,tracelengths)
    add_tracelengths(brd, *brdid, offset, tracelengths)
    add_logos(brd, side)
    
    path = f"pcb_output/{layer}{side}"
    # Check whether the specified path exists or not
    isExist = os.path.exists(path)
    if not isExist:
       # Create a new directory because it does not exist
       os.makedirs(path)
    brd.save(path.split('/')[-1], subdir=path.split('/')[0])
    #doesn't save in subfolders.  So put these in the subfolders
    import glob
    for a in list(glob.glob(path+".*"))+list(glob.glob(path+"-*"))+list(glob.glob(path+"_*")):
        os.rename(a, path+"/"+a.split("/")[-1])
    rename_files(f"pcb_output/{layer}{side}")
    print("done with layer ", layer, "side", side)

Rendering Gerber GTD...
Rendering Gerber GTP...
Rendering Gerber GTO...
Rendering Gerber GTS...
Rendering Gerber GTL...
Rendering Gerber GBL...
Rendering Gerber GBS...
Rendering Gerber GBO...
Rendering Gerber GBP...
Rendering Gerber GBD...
Rendering Gerber GML...
Rendering Gerber GP2...
Rendering Gerber GP3...
Rendering Gerber GP4...
Rendering Gerber GP5...
Rendering excellon drill files...
Rendering preview_top.['pdf']...
Rendering preview_top_docu.['pdf']...
Rendering preview_bot.['pdf']...
Rendering preview_bot_docu.['pdf']...
Rendering preview_all.['pdf']...
done with layer  5 side R
Rendering Gerber GTD...
Rendering Gerber GTP...
Rendering Gerber GTO...
Rendering Gerber GTS...
Rendering Gerber GTL...
Rendering Gerber GBL...
Rendering Gerber GBS...
Rendering Gerber GBO...
Rendering Gerber GBP...
Rendering Gerber GBD...
Rendering Gerber GML...
Rendering Gerber GP2...
Rendering Gerber GP3...
Rendering Gerber GP4...
Rendering Gerber GP5...
Rendering excellon drill files...
Rendering p